# Projeto Final Redes Neurais: Propriedades Mecânicas de Materiais

#### Trio: csv_Computeiros_sinápticos_virtuais 

#### Integrantes: Diogo Pereira de Lima Carvalho, José David Sales e Mayllon Emmanoel Pequeno

<p style="text-align: justify;"> O notebook em questão consiste no processo de otimização de hiperparâmetros da rede neural MLP. Para isso, utilizou-se o <code>optuna</code>, baseando-se em um notebook do professor Daniel Roberto Cassar [1]. Para essa otimização, 

<p style="text-align: justify;"> Abaixo, foram importadas as bibliotecas necessárias e determinadas as constantes utilizadas neste notebook.

In [1]:
import lightning as L
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
import seaborn as sns
import torch
import torch.nn as nn
import torch.optim as optim
from scipy import stats
from sklearn.metrics import root_mean_squared_error
from objetos_otimizacao_rede_neural import *
from lightning.pytorch.loggers import TensorBoardLogger
from optuna import create_study, Trial
import os

In [2]:
TAMANHO_TESTE = 0.1
SEMENTE_ALEATORIA = 0
NUM_TENTATIVAS_OTIMIZACAO = 200
NUM_EPOCAS = 20

<p style="text-align: justify;"> Abaixo, foi utilizado a instância <code>Logger</code> para guardar informações importantes sobre as métricas de desempenho do modelo que será treinado. Um treinador também é criado que será responsável pelos ciclos de treinamamento do modelo.

In [3]:
logger = TensorBoardLogger(save_dir=os.getcwd(), version=1, name="lightning_logs/")
treinador = L.Trainer(logger=False,enable_checkpointing=True,max_epochs=NUM_EPOCAS)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


<p style="text-align: justify;"> Abaixo, a função <code>build_model</code> cria uma instância do modelo, em que há 5 dados de entrada (5 atributos) e 1 dado de saída, já que se possui apenas um <i>target</i> numérico. Nisso, em cada tentativa, pode-se variar o número de camadas (entre 2 e 5), o número de neurônios por camada (de 2 a 15) e se haverá ou não viés para neurônios de cada camada. 

In [4]:
def build_model(trial):
    num_dados_de_entrada = 5
    num_dados_de_saida = 1
    n_layers = trial.suggest_int("n_layers", 2, 5)
    neuronios_camadas = []
    vieses = []
    for i in range(n_layers):
        neuronios =  trial.suggest_int(f"c{i}", 2, 15)
        bia = trial.suggest_categorical(f"bias{i}", [False, True])
        neuronios_camadas.append(neuronios)
        vieses.append(bia)
    
    minha_mlp = MLP(
        num_dados_de_entrada, list(neuronios_camadas), list(vieses), num_dados_de_saida
    )
    
    return minha_mlp

<p style="text-align: justify;"> Abaixo, é defido a função <code>objective</code>, responsável por computar a métrica RMSE para a MLP com uma validação cruzada realizada a partir da iteração abaixo.

In [5]:
def objective(trial):

    k = []
    for i in range(int(1/TAMANHO_TESTE)):
        print(i)
        dm = DataModule(TAMANHO_TESTE, SEMENTE_ALEATORIA, i)
        minha_mlp = build_model(trial)
        treinador.fit(minha_mlp, dm)

        minha_mlp.eval()
        dm.setup("test")

        with torch.no_grad():
            X_true = dm.X_teste

            y_true = dm.y_teste
            y_true = dm.y_scaler.inverse_transform(y_true)

            y_pred = minha_mlp(X_true)
            y_pred = dm.y_scaler.inverse_transform(y_pred)

            RMSE = root_mean_squared_error(y_true, y_pred)
        
        k.append(RMSE)

    rmse_medio = (sum(np.array(k)**2)/int(1/TAMANHO_TESTE))**.5
    return rmse_medio

<p style="text-align: justify;"> Por fim, criou-se a instância de estudo de <code>create_study</code>, sendo utilizado para resolver um problema de minimização.

In [6]:
%%capture
study = create_study(direction='minimize')
parametros_totais = []

for _ in range(NUM_TENTATIVAS_OTIMIZACAO):

    study.optimize(objective, n_trials=1)
    study.trials_dataframe().to_excel('triagem.xlsx')

[I 2024-05-06 01:05:22,392] A new study created in memory with name: no-name-16d0489e-d964-4b86-9f91-1b2e16da2fcc

  | Name    | Type       | Params
---------------------------------------
0 | camadas | Sequential | 235   
---------------------------------------
235       Trainable params
0         Non-trainable params
235       Total params
0.001     Total estimated model params size (MB)
`Trainer.fit` stopped: `max_epochs=20` reached.

  | Name    | Type       | Params
---------------------------------------
0 | camadas | Sequential | 235   
---------------------------------------
235       Trainable params
0         Non-trainable params
235       Total params
0.001     Total estimated model params size (MB)
`Trainer.fit` stopped: `max_epochs=20` reached.

  | Name    | Type       | Params
---------------------------------------
0 | camadas | Sequential | 235   
---------------------------------------
235       Trainable params
0         Non-trainable params
235       Total params
0.

In [16]:
melhor_trial = study.best_trial

print(f"Número do melhor trial: {melhor_trial.number}")
print(f"Parâmetros do melhor trial: {melhor_trial.params}")

Número do melhor trial: 179
Parâmetros do melhor trial: {'n_layers': 3, 'c0': 13, 'bias0': True, 'c1': 11, 'bias1': False, 'c2': 3, 'bias2': False}


In [19]:
num_dados_de_entrada = 5
num_dados_de_saida = 1

neuronios_camadas = [melhor_trial.params['c0'], melhor_trial.params['c1'],  melhor_trial.params['c2']]
vieses = [melhor_trial.params['bias0'], melhor_trial.params['bias1'],  melhor_trial.params['bias2']]

minha_mlp = MLP(
    num_dados_de_entrada, list(neuronios_camadas), list(vieses), num_dados_de_saida
)

dm = DataModule(TAMANHO_TESTE, SEMENTE_ALEATORIA, 1)

dm.setup("test")

with torch.no_grad():
    X_true = dm.X_teste

    y_true = dm.y_teste
    y_true = dm.y_scaler.inverse_transform(y_true)

    y_pred = minha_mlp(X_true)
    y_pred = dm.y_scaler.inverse_transform(y_pred)

    RMSE = mean_squared_error(y_true, y_pred, squared=False)

    print(RMSE)

424.21544777282264


c:\venv\ilumpy\lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
